In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math


movies = pd.read_table('Data/movie.metadata.tsv',names=['Wiki_ID','Freebase_ID','Name','Release_Date','Box_Office_Revenue','Length','language','Country','Genre'])
characters = pd.read_table('Data/character.metadata.tsv', names=['Wiki_ID','Freebase_ID','Release_Date','Name','Actor_Date_Of_Birth','Actor_Gender','Actor_Height','Actor_Ethnicity','Actor_Name','Actor_Age_At_Release','Freebase_Map','Freebase_Character_ID','Freebase_Actor_ID'])
#name = pd.read_csv('name.clusters.txt', sep="  ")
#summaries = pd.read_csv('plot_summaries.txt', sep=" ")

#test

In [5]:
from wikidata2df import wikidata2df

# NEW

In [46]:
US = '''
SELECT ?filmLabel ?actorLabel ?reviewLabel ?freebase_ID ?score_byLabel ?characterLabel ?methodLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?film wdt:P31 wd:Q11424;
    wdt:P495 wd:Q30;
    wdt:P646 ?freebase_ID;
}
'''

In [47]:
US_film = wikidata2df(US)
US_film

,freebase_ID,filmLabel
0,/m/0gwgn1k,That's My Boy
1,/m/04g9gd,Domino
2,/m/03qlwpd,Be Like Others
3,/m/043k86w,Nothing like the Holidays
4,/m/02x32mp,Dead like Me: Life After Death
...,...,...
38371,/m/09s6g6_,Young Dynamite
38372,/m/0djyb7h,"Happy Birthday, Gemini"
38373,/m/0crwvd1,Snipes
38374,/m/010bdwhr,Sister


In [48]:
US_rev = '''
SELECT ?filmLabel ?actorLabel ?reviewLabel ?freebase_ID ?score_byLabel ?characterLabel ?methodLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?film wdt:P31 wd:Q11424;
    wdt:P495 wd:Q30;
    wdt:P646 ?freebase_ID;
    p:P444 ?revieww.
    ?revieww ps:P444 ?review;
    pq:P447 ?score_by;
    pq:P459 ?method.
}
'''

In [49]:
US_film_rev = wikidata2df(US_rev)
US_film_rev

,reviewLabel,freebase_ID,filmLabel,methodLabel,score_byLabel
0,75%,/m/08vg7b8,The Sailor Who Fell from Grace with the Sea,Tomatometer score,Rotten Tomatoes
1,75%,/m/02sk8g,Genghis Blues,Tomatometer score,Rotten Tomatoes
2,75%,/m/0ll354z,Unconditional,Tomatometer score,Rotten Tomatoes
3,75%,/m/0b74bnh,The Chosen,Tomatometer score,Rotten Tomatoes
4,75%,/m/065y_mc,Cold Souls,Tomatometer score,Rotten Tomatoes
...,...,...,...,...,...
24103,3.4/10,/m/03cj190,Everybody Wants to Be Italian,Rotten Tomatoes average rating,Rotten Tomatoes
24104,3.4/10,/m/02z0b34,And Then Came Love,Rotten Tomatoes average rating,Rotten Tomatoes
24105,3.4/10,/m/063kdr9,Mr. Right,Rotten Tomatoes average rating,Rotten Tomatoes
24106,3.4/10,/m/02rnd2j,The Lurking Fear,Rotten Tomatoes average rating,Rotten Tomatoes


In [50]:
US_film_rev_RTav = pd.DataFrame(US_film_rev[US_film_rev.methodLabel == 'Rotten Tomatoes average rating'][['freebase_ID','reviewLabel']])
US_film_rev_RTav.columns = ['freebase_ID','RT average']
US_film_rev_Ts = pd.DataFrame(US_film_rev[US_film_rev.methodLabel == 'Tomatometer score'][['freebase_ID','reviewLabel']])
US_film_rev_Ts.columns = ['freebase_ID','TS score']
US_film_rev_Ts = US_film_rev_Ts.set_index(['freebase_ID'])
US_film_rev_RTav = US_film_rev_RTav.set_index(['freebase_ID'])
US_film_rev_RTav

,RT average
freebase_ID,
/m/05v4k9v,6.8/10
/m/0wxry4y,6.8/10
/m/04zy43q,6.8/10
/m/0b_rb7,6.8/10
/m/0fmkj_,6.8/10
...,...
/m/03cj190,3.4/10
/m/02z0b34,3.4/10
/m/063kdr9,3.4/10


In [51]:
US_film_rev_Ts

,TS score
freebase_ID,
/m/08vg7b8,75%
/m/02sk8g,75%
/m/0ll354z,75%
/m/0b74bnh,75%
/m/065y_mc,75%
...,...
/m/0kt_rf,1%
/m/073f2t,3%
/m/01tlwb,3%


In [52]:
US_film_rev_tot = US_film_rev_Ts.merge(US_film_rev_RTav, left_index=True,right_index=True, how='left')
US_film_rev_tot

,TS score,RT average
freebase_ID,,
/m/08vg7b8,75%,5.3/10
/m/02sk8g,75%,7.2/10
/m/0ll354z,75%,6.5/10
/m/0b74bnh,75%,6.4/10
/m/065y_mc,75%,6.4/10
...,...,...
/m/0kt_rf,1%,2.6/10
/m/073f2t,3%,3.2/10
/m/01tlwb,3%,2.9/10


In [64]:
df_movie = pd.read_table('Data/movie.metadata.tsv',header=None)
df_movie.columns=['Wikipedia_movie_ID' , 'Freebase_movie_ID', 'Movie_name' , 'Movie_date' , 'Movie_revenue' , 'Movie_runtime' , 'Movie_languages' , 'Movie_countries' , 'Movie_genres']
df_movie['Movie_countries'] = df_movie['Movie_countries'].apply( lambda x: list(eval(x).values()))
df_movie['Movie_genres'] = df_movie['Movie_genres'].apply( lambda x: list(eval(x).values()) )


#Movie metadata of american movies
df_movie_usa = df_movie.loc[df_movie['Movie_countries'].explode().isin(['United States of America'])[df_movie['Movie_countries'].explode().isin(['United States of America'])].index]
df_movie_usa

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_date,Movie_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Thriller, Science Fiction, Horror, Adventure,..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]"
5,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...",[United States of America],"[Short Film, Silent film, Indie, Black-and-whi..."
7,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Musical, Comedy, Black-and-white]"
10,175026,/m/017n1p,Sarah and Son,1930,NaN,86.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Drama, Black-and-white]"
...,...,...,...,...,...,...,...,...,...
81725,1918494,/m/0660qx,State and Main,2000-08-26,6944471.0,106.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","[France, United States of America]","[Parody, Americana, Comedy]"
81726,664006,/m/030xw6,Guilty as Sin,1993-06-04,22886222.0,107.0,{},[United States of America],"[Thriller, Erotic thriller, Psychological thri..."
81727,24209227,/m/07k5mlk,"The Time, the Place and the Girl",1946,NaN,105.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Comedy film, Romance Film, Musical]"
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],[Drama]


In [65]:
movies_review = df_movie_usa.merge(US_film_rev_tot, left_on = 'Freebase_movie_ID', right_index = True, how='left')
movies_review

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_date,Movie_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres,TS score,RT average
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...",22%,3.8/10
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]",NaN,NaN
5,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...",[United States of America],"[Short Film, Silent film, Indie, Black-and-whi...",NaN,NaN
7,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Musical, Comedy, Black-and-white]",80%,7/10
10,175026,/m/017n1p,Sarah and Son,1930,NaN,86.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Drama, Black-and-white]",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
81725,1918494,/m/0660qx,State and Main,2000-08-26,6944471.0,106.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","[France, United States of America]","[Parody, Americana, Comedy]",85%,7.3/10
81726,664006,/m/030xw6,Guilty as Sin,1993-06-04,22886222.0,107.0,{},[United States of America],"[Thriller, Erotic thriller, Psychological thri...",38%,5.3/10
81727,24209227,/m/07k5mlk,"The Time, the Place and the Girl",1946,NaN,105.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Comedy film, Romance Film, Musical]",NaN,NaN
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],[Drama],NaN,NaN


In [74]:
box_office = '''
SELECT ?freebase_ID ?Box_office WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?film wdt:P31 wd:Q11424;
    wdt:P495 wd:Q30;
    wdt:P646 ?freebase_ID;
    wdt:P2142 ?Box_office.
}
'''

In [84]:
wikidata2df(box_office)

,Box_office,freebase_ID
0,265198,/m/08s4sw
1,255000,/m/08y5wd
2,306224,/m/0b62y7
3,216166,/m/02pl3jt
4,295206,/m/0274ft
...,...,...
1979,312296056,/m/011r9j66
1980,334901337,/m/011djb5_
1981,334933831,/m/0v2vjxh
1982,303144152,/m/0y5297q


In [81]:
df_movie_usa[df_movie_usa.Movie_revenue.notna()]

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_date,Movie_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Thriller, Science Fiction, Horror, Adventure,..."
7,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Musical, Comedy, Black-and-white]"
17,77856,/m/0kcn7,Mary Poppins,1964-08-27,102272727.0,139.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Children's/Family, Musical, Fantasy, Comedy, ..."
21,612710,/m/02wjqm,New Rose Hotel,1999-10-01,21521.0,92.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Thriller, Science Fiction, Future noir, Indie..."
29,156558,/m/014k4y,Baby Boy,2001-06-27,29381649.0,123.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Crime Fiction, Drama, Coming of age]"
...,...,...,...,...,...,...,...,...,...
81677,6740261,/m/0glmbl,Eternally Yours,1939-10-07,683131.0,91.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Family Drama, Black-and-white, Drama, Comedy,..."
81695,54540,/m/0f7hw,Coming to America,1988-06-29,288752301.0,117.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Romantic comedy, Comedy of manners, Drama, Co..."
81720,7761830,/m/0kvgqb,Spaced Invaders,1990,15369573.0,100.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Alien Film, Science Fiction, Family Film, Com..."
81725,1918494,/m/0660qx,State and Main,2000-08-26,6944471.0,106.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","[France, United States of America]","[Parody, Americana, Comedy]"


In [5]:
from wikidata2df import wikidata2df

In [10]:
actor = '''
SELECT ?filmLabel ?actorLabel ?reviewLabel ?freebase_ID ?score_byLabel ?characterLabel ?methodLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?film wdt:P31 wd:Q11424;
    wdt:P495 wd:Q30;
    wdt:P646 ?freebase_ID;
    p:P161 ?actorr.
    ?actorr ps:P161 ?actor;
    pq:P4633 ?character.
}

'''

In [11]:
US_film_actor = wikidata2df(actor)
US_film_actor

,filmLabel,actorLabel,characterLabel,freebase_ID
0,Forrest Gump,Joe Alaskey,Richard Nixon,/m/0bdjd
1,The Matrix Reloaded,Robert Mammone,AK,/m/0x25q
2,The Matrix Revolutions,Robert Mammone,AK,/m/01dyvs
3,On the Waterfront,Rudy Bond,Moose,/m/0bmhn
4,Thor,Kelly Hawthorne,Vikings,/m/062zjtt
...,...,...,...,...
14673,Top Gun,Adrian Pasdar,"Lt. Charles ""Chipper"" Piper",/m/018js4
14674,Top Gun,Anthony Edwards,LTJG Nick 'Goose' Bradshaw,/m/018js4
14675,Top Gun,Whip Hubley,Lt. Rick 'Hollywood' Neven,/m/018js4
14676,Top Gun,Meg Ryan,Carole Bradshaw,/m/018js4


In [12]:
US_film_actor[US_film_actor.freebase_ID=='/m/0bdjd']

,filmLabel,actorLabel,characterLabel,freebase_ID
0,Forrest Gump,Joe Alaskey,Richard Nixon,/m/0bdjd
2572,Forrest Gump,Joe Stefanelli,John Lennon,/m/0bdjd
2596,Forrest Gump,Tom Hanks,Forrest Gump,/m/0bdjd
2720,Forrest Gump,Jed Gillin,John F. Kennedy,/m/0bdjd
2906,Forrest Gump,Peter Dobson,Elvis,/m/0bdjd
2951,Forrest Gump,John William Galt,Lyndon B. Johnson,/m/0bdjd
4368,Forrest Gump,Geoffrey Blake,Wesley,/m/0bdjd
5357,Forrest Gump,Siobhan Fallon Hogan,Dorothy Harris,/m/0bdjd
6176,Forrest Gump,Robin Wright,Jenny Curran,/m/0bdjd
7038,Forrest Gump,Richard D'Alessandro,Abbie Hoffman,/m/0bdjd


In [21]:
US_film_actor.freebase_ID.value_counts()

/m/0126b8kv    70
/m/0gvsynb     60
/m/02pth35     60
/m/031hcx      59
/m/031778      51
               ..
/m/03kysk       1
/m/0bxmws       1
/m/0gncc        1
/m/03qpr2       1
/m/0f2w9z       1
Name: freebase_ID, Length: 1967, dtype: int64

In [8]:
actor2 = '''
SELECT ?filmLabel ?actorLabel ?reviewLabel ?freebase_ID ?score_byLabel ?characterLabel ?methodLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?film wdt:P31 wd:Q11424;
    wdt:P495 wd:Q30;
    wdt:P646 ?freebase_ID;
    p:P161 ?actorr.
    ?actorr ps:P161 ?actor;
    pq:P4633 ?character.
}

'''

In [9]:
wikidata2df(actor2)

,filmLabel,actorLabel,characterLabel,freebase_ID
0,Forrest Gump,Joe Alaskey,Richard Nixon,/m/0bdjd
1,The Matrix Reloaded,Robert Mammone,AK,/m/0x25q
2,The Matrix Revolutions,Robert Mammone,AK,/m/01dyvs
3,On the Waterfront,Rudy Bond,Moose,/m/0bmhn
4,Thor,Kelly Hawthorne,Vikings,/m/062zjtt
...,...,...,...,...
14673,Top Gun,Adrian Pasdar,"Lt. Charles ""Chipper"" Piper",/m/018js4
14674,Top Gun,Anthony Edwards,LTJG Nick 'Goose' Bradshaw,/m/018js4
14675,Top Gun,Whip Hubley,Lt. Rick 'Hollywood' Neven,/m/018js4
14676,Top Gun,Meg Ryan,Carole Bradshaw,/m/018js4


In [35]:
actor_rank = '''
SELECT ?filmLabel ?actorLabel ?reviewLabel ?freebase_ID ?score_byLabel ?rankLabel ?roleLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?film wdt:P31 wd:Q11424;
    wdt:P495 wd:Q30;
    wdt:P646 ?freebase_ID;
    p:P161 ?actorr.
    ?actorr ps:P161 ?actor;
    pq:P4633 ?role;
    pq:P1545 ?rank.
}

'''

In [41]:
rank = wikidata2df(actor_rank)
rank

,freebase_ID,filmLabel,roleLabel,rankLabel,actorLabel
0,/m/062zjtt,Thor,Vikings,46,Kelly Hawthorne
1,/m/062zjtt,Thor,Vikings,43,Hilary Pingle
2,/m/062zjtt,Thor,Vikings,44,Shawn-Caulin Young
3,/m/062zjtt,Thor,Vikings,45,Kinsey McLean
4,/m/0b7582l,Heart Beat,Dick,7,Tony Bill
...,...,...,...,...,...
1645,/m/0gxw8cp,Lies in Plain Sight,Young Sofia,5,Daniela Bobadilla
1646,/m/0gxw8cp,Lies in Plain Sight,Doctor Stone,7,Lupe Ontiveros
1647,/m/04p4b6r,The Autobiography of a Flea,Aged Priest,12,John Tate
1648,/m/03cvx8t,Sunshine Cleaning,Rose Lorkowski,1,Amy Adams


In [31]:
rank.freebase_ID.value_counts()

/m/0bc1yhb     104
/m/0126b8kv    100
/m/0125zrjx     97
/m/0cc846d      93
/m/0n15g8q      66
              ... 
/m/048tv9        1
/m/0h21v2        1
/m/03cw4fs       1
/m/03cvypj       1
/m/0sxlb         1
Name: freebase_ID, Length: 76, dtype: int64